#### openai 모델에 sql 생성 요청하기

In [2]:
def make_prompt(ddl, question, query=''):
    prompt = f"""당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.
    ### DDL :
    {ddl}

    ### Question :
    {question}

    ### SQL :
    {query}"""

    return prompt

In [ ]:
# 요청 제한(rate limit)을 관리하면서 비동기적으로 요청을 보낼 수 있음
# https://github.com/openai/openai-cookbook/blob/main/examples/api_request_parallel_processor.py

In [ ]:
from pathlib import Path
import json

def make_requests_for_gpt_evaluation(df, filename, dir='requests'):
    if not Path(dir).exists():
        Path(dir).mkdir(parents=True)
    prompts = []
    for idx, row in df.iterrows():
        prompts.append("""Based on below DDL and Question, evaluate gen_sql can resolve Question. \
                       If gen_sql and gt_sql do equal job, return "yes" else return "no". Output JSON Format: {"resolve_yn": ""}"""\
                        + f"""
### DDL : {row['ddl']}
### Question : {row['question']}
### gt_sql : {row['gt_sql']}
### gen_sql : {row['gen_sql']}"""
)

    jobs = [
        {
            "model": "gpt-4o-mini",
            "response_format": {
                "type": "json_object"
                },
            "messages": [
                {"role": "system", "content": prompt}
            ]
        }
        for prompt in prompts
    ]

    with open(Path(dir, filename), 'w') as f:
        for job in jobs:
            json_string = json.dumps(job)
            f.write(json_string + "\n")

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = ""

# python api_request_parallel_processor.py \
#   --requests_filepath requests/gpt-4o-mini_requests.jsonl
#   --save_filepath results/gpt-4o-mini_results.jsonl
#   --request_url https://api.openai.com/v1/chat/completions
#   --api_key $OPENAI_API_KEY
#   --max_requests_per_minute 100
#   --max_tokens_per_minute 100000
#   --token_encoding_name cl100k_base
#   --max_attempts 3
#   --logging_level 20

#### 결과를 csv 로 변환

In [ ]:
import pandas as pd

def change_jsonl_to_csv(input_file, output_file, prompt_column="prompt", response_column="response"):
    prompts = []
    responses = []

    with open(input_file, 'r') as f:
        for data in f:
            prompts.append(json.loads(data)[0]['messages'][0]['content'])
            responses.append(json.loads(data)[1]['choices'][0]['message']['content'])

    df = pd.DataFrame({prompt_column: prompts, response_column: responses})
    df.to_csv(output_file, index=False)

    return df

#### === 성능 평가 파이프라인 준비 완료 ===

#### 실습: 미세 조정 수행하기

In [ ]:
# 기초 모델로 생성하기

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

def make_inference_pipeline(model_id):
    tokenizer = AutoTokenizer.from_pretrained(model_id)
    model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto", load_in_4bit=True, bnb_4bit_compute_dtype=torch.float16)
    pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)
    return pipe

model_id = "beomi/Yi-Ko-6B"
hf_pipe = make_inference_pipeline(model_id)

example = """당신은 SQL을 생성하는 SQL 봇입니다. DDL의 테이블을 활용한 Question을 해결할 수 있는 SQL 쿼리를 생성하세요.

### DDL :
CREATE TABLE players (
    player_id INT PRIMARY KEY AUTO_INCREMENT,
    username VARCHAR(255) UNIQUE NOT NULL,
    email VARCHAR(255) UNIQUE NOT NULL,
    password_hash VARCHAR(255) NOT NULL,
    date_joined DATETIME NOT NULL,
    last_login DATETIME
);

### Question :
사용자 이름에 'admin'이 포함되어 있는 계정의 수를 알려주세요.

### SQL :
"""

hf_pipe(example, do_sample=False, return_full_text=False, max_length=1024, truncation=True)


In [ ]:
# 기초 모델 성능 측정
from datasets import load_dataset

df = load_dataset("shangrilar/ko_text2sql", "origin")['test']
df = df.to_pandas()
for idx, row in df.iterrows():
    prompt = make_prompt(row['context'], row['question'])
    df.loc[idx, 'prompt'] = prompt

# sql 생성
gen_sqls = hf_pipe(df['prompt'].tolist(), do_sample=False,
                   return_full_text=False, max_length=1024, truncation=True)
gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)

# GPT-4 평가 수행
!python api_request_parallel_processor.py \
    --requests_filepath requests/{eval_filepath} \
    --save_filepath results/{eval_filepath} \
    --request_url https://api.openai.com/v1/chat/completions \
    --api_key $OPENAI_API_KEY \
    --max_requests_per_minute 2500 \
    --max_tokens_per_minute 100000 \
    --token_encoding_name cl100k_base \
    --max_attempts 5 \
    --logging_level 20


In [ ]:
# 미세 조정 수행
# 학습 데이터 불러오기

from datasets import load_dataset

df_sql = load_dataset("shangrilar/ko_text2sql", "origin")['train']
df_sql = df_sql.to_pandas()
df_sql = df_sql.dropna().sample(frac=1, random_state=42)
df_sql = df_sql.query("db_id != 1") # 데이터셋에서 평가에 사용하기로 한 db_id 가 1인 데이터는 제거한다

for idx, row in df_sql.iterrows():
    df_sql.loc[idx, 'text'] = make_prompt(row['context'], row['question'], row['answer'])

!mkdir data
df_sql.to_csv('data/train.csv', index=False)

In [ ]:
# autotrain-advanced 라이브러리를 사용해 지도 미세 조정을 수행함

base_model = 'beomi/Yi-Ko-6B'
finetuned_model = 'yi-ko-6b-text2sql'

# !autotrain llm \
#  --train \
#  --model {base_model} \
#  --project-name {finetuned_model} \
#  --data-path data/ \
#  --text-column text \
#  --lr 2e-4 \
#  --batch-size 8 \
#  --epochs 1 \
#  --block-size 1024 \
#  --warmup-ratio 0.1 \
#  --lora-r 16 \
#  --lora-alpha 32 \
#  --lora-dropout 0.05 \
#  --weight-decay 0.01 \
#  --gradient-accumulation 8 \
#  --mixed-precision fp16 \
#  --use-peft \
#  --quantization int4 \
#  --trainer sft

In [ ]:
# LoRA 어댑터 결합 및 허깅페이스 허브 업로드
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, PeftModel

model_name = start_model
device_map = {"": 0}

# LoRA와 기초 모델 파라미터 합치기
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map
    )

model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# 토크나이저 설정   
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# 허깅페이스 허브에 모델 및 토크나이저 저장
model.push_to_hub(new_model, use_temp_dir=False)
tokenizer.push_to_hub(new_model, use_temp_dir=False)

In [ ]:
# 미세 조정한 모델로 예시 데이터에 대한 SQL 생성

model_id = "shangrilar/yi-ko-6b-text2sql"
hf_pipe = make_inference_pipeline(model_id)

hf_pipe(example, do_sample=False, return_full_text=False, max_length=1024, truncation=True)

In [ ]:
# 미세 조정한 모델 성능 측정
# sql 생성 수행
gen_sqls = hf_pipe(df['prompt'].tolist(),
                   do_sample=False,
                   return_full_text=False,
                   max_length=1024,
                   truncation=True)

gen_sqls = [x[0]['generated_text'] for x in gen_sqls]
df['gen_sql'] = gen_sqls

# 평가를 위한 requests.jsonl 생성
eval_filepath = "text2sql_evaluation_finetuned.jsonl"
make_requests_for_gpt_evaluation(df, eval_filepath)

# GPT-4 평가 수행
# !python api_request_parallel_processor.py \
#     --requests_filepath requests/{eval_filepath} \
#     --save_filepath results/{eval_filepath} \
#     --request_url https://api.openai.com/v1/chat/completions \
#     --api_key $OPENAI_API_KEY \
#     --max_requests_per_minute 2500 \
#     --max_tokens_per_minute 100000 \
#     --token_encoding_name cl100k_base \
#     --max_attempts 5 \
#     --logging_level 20

In [ ]:
# Yi-Ko-6B 보다 두 배 더 큰 모델
# base_model = 'beomi/OPEN-SOLAR-KO-10.7B'

In [ ]:
# 추가 연습문제

## 1. 평가 데이터셋 늘리기
## 2. 오류 분석과 데이터셋 추가 : 모델이 잘 생성하지 못하는 SQL 패턴이 있는지 분석, 학습 데이터셋 추가
## 3. DPO 학습 활용 : GPT-4 가 생성한 결과를 chosen 세트, 기초 모델이나 미세 조정 모델이 생성한 결과를 rejected 세트로 사용
## 4. 다른 모델 활용 : Code Llama ? (Code 생성에 특화된 모델인 만큼 코드와 유사한 SQL 생성도 잘 하지 않을까?)